In [1]:
'''
pip install requests
pip install beautifulsoup4
pip install lxml
pip install tqdm
'''
import requests
from bs4 import BeautifulSoup
from lxml import html
import os
from urllib.parse import urljoin
import re
from tqdm import tqdm 

In [2]:
# URL de la página web 
url = 'https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page'

# Realiza una solicitud GET para obtener el contenido de la página
response = requests.get(url)

In [3]:
 # Directorio donde guardarás los archivos descargados
directorio_destino = 'datasets/'
# Valores a buscar
valores_a_buscar = ['2023-06', '2022-06']

In [4]:
# Verifica si la solicitud se completó exitosamente
if response.status_code == 200:
    # Analiza el contenido de la página con BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encuentra todos los elementos <a> con el atributo href
    enlaces = soup.find_all('a', href=True)
  

    # Crea el directorio de destino si no existe
    os.makedirs(directorio_destino, exist_ok=True)

    # Itera a través de los enlaces y descarga los archivos que cumplen con el criterio
    for enlace in enlaces:
        url_relativa = enlace['href']
        if url_relativa.startswith(('http', 'https')):
            # URL absoluta, no hacer nada
            url_absoluta = url_relativa
        else:
            # URL relativa, construir URL absoluta
            url_absoluta = urljoin(url, url_relativa)
        # Construye la expresión regular dinámica
        expresion_regular = '|'.join(map(re.escape, valores_a_buscar))
        # Verifica si la URL absoluta contiene "2023-06" o "2022-06"
        if re.search(expresion_regular, url_absoluta):
            # Extrae el nombre del archivo de la URL
            nombre_archivo = url_relativa.split('/')[-1]

            # Descarga el archivo con barra de progreso
            response = requests.get(url_absoluta, stream=True)

            if response.status_code == 200:
                ruta_guardado = os.path.join(directorio_destino, nombre_archivo)
                total_size = int(response.headers.get('content-length', 0))
                with tqdm(total=total_size, unit='B', unit_scale=True, unit_divisor=1024) as pbar:
                    with open(ruta_guardado, 'wb') as archivo:
                        for data in response.iter_content(chunk_size=1024):
                            pbar.update(len(data))
                            archivo.write(data)
                print("Archivo descargado:", nombre_archivo)
            else:
                print("No se pudo descargar el archivo:", nombre_archivo)

else:
    print("Error al acceder a la página:", response.status_code)

100%|██████████| 52.5M/52.5M [00:11<00:00, 4.70MB/s]


Archivo descargado: yellow_tripdata_2023-06.parquet


100%|██████████| 1.49M/1.49M [00:01<00:00, 1.53MB/s]


Archivo descargado: green_tripdata_2023-06.parquet


100%|██████████| 13.4M/13.4M [00:03<00:00, 3.79MB/s]


...

Archivo descargado: green_tripdata_2022-06.parquet


100%|██████████| 11.7M/11.7M [00:02<00:00, 4.58MB/s]


Archivo descargado: fhv_tripdata_2022-06.parquet


100%|██████████| 437M/437M [01:29<00:00, 5.14MB/s] 

Archivo descargado: fhvhv_tripdata_2022-06.parquet
